Federated Learning using PyTorch and PySyft with Trusted FedAvg based on https://arxiv.org/pdf/2104.07853.pdf

## Install PySyft and Torchvision

In [ ]:
!pip install syft==0.2.9

In [ ]:
!pip install torchvision==0.2.0

## FL on MNIST dataset

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import logging

from numpy.linalg import norm
from statistics import median

# import Pysyft to help us to simulate federated learning
import syft as sy

# hook PyTorch to PySyft, i.e. add extra functionalities to support Federated Learning and other private AI tools
hook = sy.TorchHook(torch) 

# create clients
clients = []
clients.append(sy.VirtualWorker(hook, id="bob"))
clients.append(sy.VirtualWorker(hook, id="alice"))
clients.append(sy.VirtualWorker(hook, id="untrustful"))

In [ ]:
# define the args
args = {
    'use_cuda' : True,
    'batch_size' : 64,
    'test_batch_size' : 1000,
    'lr' : 0.01,
    'log_interval' : 10,
    # with federated learning convergence is faster and less epochs are needed
    'epochs' : 7
}

# check to use GPU or not
use_cuda = args['use_cuda'] and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [ ]:
# create a simple CNN net
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels = 1, out_channels = 32, kernel_size = 3, stride = 1),
            nn.ReLU(),
            nn.Conv2d(in_channels=32,out_channels = 64, kernel_size = 3, stride = 1),
            nn.ReLU()
        )
        
        self.fc = nn.Sequential(
            nn.Linear(in_features=64*12*12, out_features=128),
            nn.ReLU(),
            nn.Linear(in_features=128, out_features=10),
        )

        self.dropout = nn.Dropout2d(0.25)
    
    def forward(self, x):
        x = self.conv(x)
        x = F.max_pool2d(x,2)
        x = x.view(-1, 64*12*12)
        x = self.fc(x)
        x = F.log_softmax(x, dim=1)
        return x

In [ ]:
# prepare and distribute the data across workers
# normally there is no need to distribute data, since it is already at the clients
# this is more of a simulation of federated learning
federated_dataset = datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])).federate(tuple(clients))

federated_train_loader = sy.FederatedDataLoader(federated_dataset, batch_size=args['batch_size'], shuffle=True)

# test data remains at the central entity
test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=args['test_batch_size'], shuffle=True)

In [ ]:
# classic torch code for training except for the federated part
def train_locally(args, models, device, train_loader, optimizers, epoch):
    for c, m in models.items():
        m.train()
        # send models to workers
        m.send(c)

    # iterate over federated data
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        optimizers[data.location].zero_grad()
        output = models[data.location](data)
        # loss is a ptr to the tensor loss at the remote location
        loss = F.nll_loss(output, target)
        # call backward() on the loss ptr, that will send the command to call
        # backward on the actual loss tensor present on the remote machine
        loss.backward()
        optimizers[data.location].step()

        if batch_idx % args['log_interval'] == 0:

            # get back loss, that was created at remote worker
            loss = loss.get()

            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tWorker: {}'.format(
                    epoch, 
                    batch_idx * args['batch_size'], # no of images done
                    len(train_loader) * args['batch_size'], # total images left
                    100. * batch_idx / len(train_loader),
                    loss,
                    data.location.id
                )
            )
    
    # get back models for aggregation
    for m in models.values():
        m = m.get()

In [ ]:
# classic torch code for testing
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)

            # add losses together
            test_loss += F.nll_loss(output, target, reduction='sum').item() 

            # get the index of the max probability class
            pred = output.argmax(dim=1, keepdim=True)  
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
def aggregate(central_model, models, weights, trust):
    # firstly compute new weight values
    with torch.no_grad():
        dataXtrust = 0
        for c in models:
            weights['conv0_mean_weight'] += models[c].conv[0].weight.data.clone()*len(federated_dataset.__getitem__(c.id))*trust[c]
            weights['conv0_mean_bias'] += models[c].conv[0].bias.data.clone()*len(federated_dataset.__getitem__(c.id))*trust[c]
            weights['conv2_mean_weight'] += models[c].conv[2].weight.data.clone()*len(federated_dataset.__getitem__(c.id))*trust[c]
            weights['conv2_mean_bias'] += models[c].conv[2].bias.data.clone()*len(federated_dataset.__getitem__(c.id))*trust[c]
            weights['fc0_mean_weight'] += models[c].fc[0].weight.data.clone()*len(federated_dataset.__getitem__(c.id))*trust[c]
            weights['fc0_mean_bias'] += models[c].fc[0].bias.data.clone()*len(federated_dataset.__getitem__(c.id))*trust[c]
            weights['fc2_mean_weight'] += models[c].fc[2].weight.data.clone()*len(federated_dataset.__getitem__(c.id))*trust[c]
            weights['fc2_mean_bias'] += models[c].fc[2].bias.data.clone()*len(federated_dataset.__getitem__(c.id))*trust[c]

            dataXtrust += len(federated_dataset.__getitem__(c.id))*trust[c]

        weights['conv0_mean_weight'] = weights['conv0_mean_weight']/dataXtrust
        weights['conv0_mean_bias'] = weights['conv0_mean_bias']/dataXtrust
        weights['conv2_mean_weight'] = weights['conv2_mean_weight']/dataXtrust
        weights['conv2_mean_bias'] = weights['conv2_mean_bias']/dataXtrust
        weights['fc0_mean_weight'] = weights['fc0_mean_weight']/dataXtrust
        weights['fc0_mean_bias'] = weights['fc0_mean_bias']/dataXtrust
        weights['fc2_mean_weight'] = weights['fc2_mean_weight']/dataXtrust
        weights['fc2_mean_bias'] = weights['fc2_mean_bias']/dataXtrust

        # then copy them to the local models
        for m in models.values():
            m.conv[0].weight.data = weights['conv0_mean_weight'].data.clone()
            m.conv[0].bias.data = weights['conv0_mean_bias'].data.clone()
            m.conv[2].weight.data = weights['conv2_mean_weight'].data.clone()
            m.conv[2].bias.data = weights['conv2_mean_bias'].data.clone()
            m.fc[0].weight.data = weights['fc0_mean_weight'].data.clone()
            m.fc[0].bias.data = weights['fc0_mean_bias'].data.clone()
            m.fc[2].weight.data = weights['fc2_mean_weight'].data.clone()
            m.fc[2].bias.data = weights['fc2_mean_bias'].data.clone()

        # and to the central model for the test set
        central_model.conv[0].weight.data = weights['conv0_mean_weight'].data.clone()
        central_model.conv[0].bias.data = weights['conv0_mean_bias'].data.clone()
        central_model.conv[2].weight.data = weights['conv2_mean_weight'].data.clone()
        central_model.conv[2].bias.data = weights['conv2_mean_bias'].data.clone()
        central_model.fc[0].weight.data = weights['fc0_mean_weight'].data.clone()
        central_model.fc[0].bias.data = weights['fc0_mean_bias'].data.clone()
        central_model.fc[2].weight.data = weights['fc2_mean_weight'].data.clone()
        central_model.fc[2].bias.data = weights['fc2_mean_bias'].data.clone()

In [ ]:
def computeTrust(models, trust, r, s):
    # dev[i] shows how the weights of model of client i differ from the models of all other clients
    dev = [0 for i in clients]
    for n, i in enumerate(clients):
        for j in clients:
            dev[n] += norm(models[j].conv[0].weight.data.cpu()-models[i].conv[0].weight.data.cpu())**2
            dev[n] += norm(models[j].conv[2].weight.data.cpu()-models[i].conv[2].weight.data.cpu())**2
            dev[n] += norm(models[j].fc[0].weight.data.cpu()-models[i].fc[0].weight.data.cpu())**2
            dev[n] += norm(models[j].fc[2].weight.data.cpu()-models[i].fc[2].weight.data.cpu())**2
        dev[n] /= len(clients)

    # I[i] = 1 if client i acts normally and 0 if malicious or malfunctions
    I = [1 if d <= 1.3 * median(sorted(dev)) else 0 for d in dev]
    #print("dev: ",dev) # testing
    #print("median*1.3: ", 1.3*median(dev)) # testing
    #print("I: ", I) # testing

    for i in range(len(clients)):
        r[i] = 0.8*r[i] + I[i]
        s[i] = 3.0*s[i] + 1 - I[i]

    for i, c in enumerate(clients):
        trust[c] = (r[i]+1)/(r[i]+s[i]+2)

In [ ]:
# central model
central_model = Net().to(device)
# optimizer for central model not needed if model is not trained
#optimizer = optim.SGD(central_model.parameters(), lr=args['lr'])
# clients models and optimizers
models = {i:Net().to(device) for i in clients}
optimizers = {i:optim.SGD(models[i].parameters(), lr=args['lr']) for i in clients}

# initialization of dictionary for model aggregation
weights = {'conv0_mean_weight' : torch.zeros(size=central_model.conv[0].weight.shape).to(device),
           'conv0_mean_bias' : torch.zeros(size=central_model.conv[0].bias.shape).to(device),
           'conv2_mean_weight' : torch.zeros(size=central_model.conv[2].weight.shape).to(device),
           'conv2_mean_bias' : torch.zeros(size=central_model.conv[2].bias.shape).to(device),
           'fc0_mean_weight' : torch.zeros(size=central_model.fc[0].weight.shape).to(device),
           'fc0_mean_bias' : torch.zeros(size=central_model.fc[0].bias.shape).to(device),
           'fc2_mean_weight' : torch.zeros(size=central_model.fc[2].weight.shape).to(device),
           'fc2_mean_bias' : torch.zeros(size=central_model.fc[2].bias.shape).to(device)}

# trust values
trust = {i:0 for i in clients}
r = [0 for i in clients]
s = [0 for i in clients]

logging.info("Starting training!")

for epoch in range(1, args['epochs'] + 1):
    train_locally(args, models, device, federated_train_loader, optimizers, epoch)
    # we intentionally change the weights of a client's model to make that client behave as untrustful
    models[clients[2]].conv[0].weight.data = models[clients[2]].conv[0].weight.data*10
    models[clients[2]].conv[2].weight.data = models[clients[2]].conv[2].weight.data*10
    models[clients[2]].fc[0].weight.data = models[clients[2]].fc[0].weight.data*10
    models[clients[2]].fc[2].weight.data = models[clients[2]].fc[2].weight.data*10
    computeTrust(models, trust, r, s)
    aggregate(central_model, models, weights, trust)
    test(central_model, device, test_loader)

Train Epoch: 1 [0/60032 (0%)]	Loss: 2.313279	Worker: bob
Train Epoch: 1 [640/60032 (1%)]	Loss: 2.202435	Worker: bob
Train Epoch: 1 [1280/60032 (2%)]	Loss: 2.088405	Worker: bob
Train Epoch: 1 [1920/60032 (3%)]	Loss: 1.774398	Worker: bob
Train Epoch: 1 [2560/60032 (4%)]	Loss: 1.459744	Worker: bob
Train Epoch: 1 [3200/60032 (5%)]	Loss: 1.063543	Worker: bob
Train Epoch: 1 [3840/60032 (6%)]	Loss: 0.717759	Worker: bob
Train Epoch: 1 [4480/60032 (7%)]	Loss: 0.649427	Worker: bob
Train Epoch: 1 [5120/60032 (9%)]	Loss: 0.635050	Worker: bob
Train Epoch: 1 [5760/60032 (10%)]	Loss: 0.571844	Worker: bob
Train Epoch: 1 [6400/60032 (11%)]	Loss: 0.415664	Worker: bob
Train Epoch: 1 [7040/60032 (12%)]	Loss: 0.454812	Worker: bob
Train Epoch: 1 [7680/60032 (13%)]	Loss: 0.593601	Worker: bob
Train Epoch: 1 [8320/60032 (14%)]	Loss: 0.470521	Worker: bob
Train Epoch: 1 [8960/60032 (15%)]	Loss: 0.392465	Worker: bob
Train Epoch: 1 [9600/60032 (16%)]	Loss: 0.397079	Worker: bob
Train Epoch: 1 [10240/60032 (17%)]	Lo